In [1]:
import os
import json
import fiona
import geemap
import numpy as np
import pandas as pd
import geopandas as gpd
import xml.etree.ElementTree as ET

from collections import Counter

from shapely.geometry import shape
from shapely.geometry import Point

# Gee and EE 
import ee

# Clustering (best for testing** due to speed)
from sklearn.cluster import KMeans

# tif file creation
import rasterio
from rasterio.transform import from_origin
from rasterio.features import rasterize

# Plotting and Vis 
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import re

In [2]:
# Initialise
ee.Authenticate()
ee.Initialize(project="jameswilliamchamberlain")

In [3]:
# basemap 
basemap_url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'

# centeral point of Samarra 
df_sites = pd.DataFrame({
    "longitude": [43.823543],
    "latitude": [34.340989],
    "name": ["Samarra Archaeological City"],
    "category": ["Cultural"],
    "date inscribed": ["2007"],
    "region": ["Arab States"],
    "url": ["https://whc.unesco.org/en/list/276"],
    "iso": [["IQ"]]
})

# Chunks of Samarra Archaeological City 
with fiona.open("chunks_new.shp") as src:
    chunks = gpd.GeoDataFrame.from_features(src, crs=src.crs)

In [4]:
## TEMP REMOVE ADDITIONAL STUFF FOR QUICKER TESTING 
# tile_43641125_34108721_43915744_34336837 only 

chunks = chunks[chunks['file_name'] == 'tile_43641125_34108721_43915744_34336837']

In [5]:
m = geemap.Map()

if df_sites.empty:
    print("No sites found for the specified URL.")
else:
    m.add_points_from_xy(df_sites, x="longitude", y="latitude", layer_name="Sites")
    center_points = df_sites[['longitude', 'latitude']].mean().values
    m.setCenter(center_points[0], center_points[1], 10)

m.add_basemap(basemap_url, name="Google Satellite", attribution="Google")

# add chunks from aoi 
m.add_gdf(chunks, layer_name="AOI", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})

m

Map(center=[34.340989, 43.823543], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

In [6]:
def create_subregions(chunks, sift_percentage_lon=0.5, sift_percentage_lat=0.415):
    """
        Shifts all polygons to 8 positions based on half the length in longitude and latitude, to create subregions - one for each direction from the center.

        Assumes aoi contains all similar polygons, and are similar to a square.

        Parameters:
            aoi (GeoDataFrame): The area of interest containing geometries.
            sift_percentage (float): Percentage of the length to shift the center point. Default is 0.5 (50%) for half the length.
    """

    if chunks.empty:
        return {}
    
    first_polygon = chunks.geometry.iloc[0]

    # take top two points of the polygon and get the length between them
    top_points = first_polygon.exterior.coords[:2]
    length_lon = abs(top_points[0][0] - top_points[1][0])

    # calculate the shift amount
    shift_amount_lon = length_lon * sift_percentage_lon 
    shift_amount_lat = length_lon * sift_percentage_lat

    shift_directions = {
        "left": (-shift_amount_lon, 0),
        "right": (shift_amount_lon, 0),
        "up": (0, shift_amount_lat),
        "down": (0, -shift_amount_lat),
        "top_left": (-shift_amount_lon, shift_amount_lat),
        "top_right": (shift_amount_lon, shift_amount_lat),
        "bottom_left": (-shift_amount_lon, -shift_amount_lat),
        "bottom_right": (shift_amount_lon, -shift_amount_lat),
    }

    subregions = []
        
    # Create subregions by shifting the geometries in all directions
    for _, (dx, dy) in shift_directions.items():
        gdf_shifted = chunks.copy()
        gdf_shifted["geometry"] = gdf_shifted["geometry"].translate(dx, dy)
        subregions.append(gdf_shifted)

    # Combine all into one GeoDataFrame
    subregions = pd.concat(subregions, ignore_index=True)
    return gpd.GeoDataFrame(subregions, crs=chunks.crs)

def clip(chunks, aoi):
    """
        Clips the chunks or subregions to the area of interest (aoi).

        Parameters:
            chunks (GeoDataFrame):      The chunks or subregions to be clipped.
            aoi (GeoDataFrame):         The area of interest (aoi) to clip the chunks against.
    """

    # clip to aoi 
    if chunks.empty or aoi.empty:
        return gpd.GeoDataFrame(columns=chunks.columns.tolist(), crs=chunks.crs)
    clipped = gpd.clip(chunks, aoi)
    clipped = clipped[clipped.geometry.notnull()]  # Remove any null geometries

    return clipped.reset_index(drop=True)

subregions = create_subregions(chunks)
subregions = clip(subregions, aoi=chunks.dissolve())

# plot as one 
m.add_gdf(subregions, layer_name="Subregions", style={"color": "blue", "fillColor": "blue", "fillOpacity": 0.1})

# Reference Points

In [7]:
def collect_points_from_geemap(map_obj, label):
    """
    Collect all drawn point features from a geemap.Map that uses ee.Feature objects,
    and return them as a GeoDataFrame with a label.

    Parameters:
        map_obj (geemap.Map): The interactive map.
        label (str): Label to assign to all collected points.

    Returns:
        GeoDataFrame: With geometry and 'label' columns.
    """

    features = map_obj.draw_features

    if not features:
        return gpd.GeoDataFrame(columns=["geometry", "label"], geometry="geometry")

    points = []
    for f in features:
        try:
            geom = f.geometry()  # call the method
            if geom.getInfo()["type"] == "Point":
                coords = geom.coordinates().getInfo()  # [lon, lat]
                points.append(Point(coords))
        except Exception as e:
            print("Skipping feature due to error:", e)

    if not points:
        return gpd.GeoDataFrame(columns=["geometry", "label"], geometry="geometry")

    gdf = gpd.GeoDataFrame(geometry=points)
    gdf["label"] = label
    gdf.set_crs("EPSG:4326", inplace=True)
    return gdf

def gen_basemap(basemap_url=None, aoi=gpd.GeoDataFrame(), polygons=None):
    """
        Generates a basemap with the specified URL and adds polygons if provided.

        Parameters:
            basemap_url (str): The URL of the basemap to be used.
            polygons (GeoDataFrame, optional): Polygons to be added to the map. Defaults to None.
            aoi (GeoDataFrame): The area of interest to be displayed on the map. (can be a set of polygons or a single polygon)
        
        Returns:
            geemap.Map: A geemap map object with the specified basemap and polygons.
    """
    m = geemap.Map()

    if not basemap_url:
        basemap_url = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'
        
    m.add_basemap(basemap_url, name="Google Satellite", attribution="Google")

    # center on aoi
    if not aoi.empty:
        m.add_gdf(aoi, layer_name="AOI", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})
        center_points = aoi.geometry.unary_union.centroid.coords[0]
        m.setCenter(center_points[0], center_points[1], 10)
    
    if polygons is not None:
        m.add_gdf(polygons, layer_name="Polygons", style={"color": "red", "fillColor": "red", "fillOpacity": 0.1})
    
    return m

In [8]:
# Example of 3 maps for urban, agricultrual, water and waste
urban_map = gen_basemap(basemap_url, aoi=chunks)
agricultural_map = gen_basemap(basemap_url, aoi=chunks)
water_map = gen_basemap(basemap_url, aoi=chunks)
wasteland_map = gen_basemap(basemap_url, aoi=chunks)
# wasteland_map = geemap.Map()

In [9]:
urban_map

Map(center=[34.22277923307027, 43.77843455433748], controls=(WidgetControl(options=['position', 'transparent_b…

In [10]:
agricultural_map

Map(center=[34.22277923307027, 43.77843455433748], controls=(WidgetControl(options=['position', 'transparent_b…

In [11]:
# water_map = water_waste_map
water_map

Map(center=[34.22277923307027, 43.77843455433748], controls=(WidgetControl(options=['position', 'transparent_b…

In [12]:
wasteland_map

Map(center=[34.22277923307027, 43.77843455433748], controls=(WidgetControl(options=['position', 'transparent_b…

In [13]:
additional_map = gen_basemap(basemap_url, aoi=chunks)
additional_map


Map(center=[34.22282359393232, 43.778616017576226], controls=(WidgetControl(options=['position', 'transparent_…

In [14]:
additional_points = collect_points_from_geemap(additional_map, label="Urban")
additional_points

,geometry,label


In [15]:
# convert to labelled points in GeoDataFrame
water_points = collect_points_from_geemap(water_map, label="Water")
agricultural_points = collect_points_from_geemap(agricultural_map, label="Agricultural")
urban_points = collect_points_from_geemap(urban_map, label="Urban")
wasteland_map = collect_points_from_geemap(wasteland_map, label="Wasteland")

# merge into one GeoDataFrame
points_list = [water_points, agricultural_points, urban_points, wasteland_map]
labels = pd.concat(points_list, ignore_index=True)


# Clustering
Here we create a Sparse matrix

then with the sparse matrix compare and label based on that

In [16]:
pth = "data/"

# collect paths for all csv files in the folder
paths = []

for root, dirs, files in os.walk(pth):
    for file in files:
        if file.endswith(".csv"):
            paths.append(os.path.join(root, file))

print(f"Found {len(paths)} CSV files.")

# select 2024 
path = [p for p in paths if "2019" in p]
path


Found 6 CSV files.


['data/2019.csv']

In [17]:
def prep_data(dir):
    """Creates two dataframes from a CSV file, one ready for clustering with additional columns removed and the other with all columns left intact."""
    df1 = pd.read_csv(dir)
    df1 = df1.dropna()
    df2_clear = df1.copy()
    df2_clear = df2_clear.drop(columns=["system:index", ".geo"])
    df2_clear = df2_clear.set_index("file_name")
    df1 = df1.set_index("file_name")
    df2_clear = df2_clear.apply(pd.to_numeric, errors='coerce')

    return df1, df2_clear

df1, df2_clear = prep_data(path[0])

mapping = {
            None: 0,
            np.NaN: 0,
            "NaN": 0,
            "None": 0,
            "Water": 1,
            "Agricultural": 2,
            "Urban": 3,
            "Wasteland": 4,
        }

In [18]:
import cluster as cl 

In [ ]:
cluster = cl.Cluster(chunks, df1, points=labels, mapping=mapping, passes=6)
cluster.reload_state(filname_prefix="temp_test/test3", filename_postfix="_state")

In [91]:
import ee
import geemap
from ipyleaflet import CircleMarker
import ipywidgets as widgets
import pandas as pd 

In [ ]:
label_list = ['Urban', 'Agricultural', 'Water', 'Wasteland', 'Wetland', 'Other']
date_range = [i for i in range(2019, 2025)] 

collection_name = "COPERNICUS/S2_SR_HARMONIZED"

with fiona.open("aoi.geojson") as src:
    aoi = gpd.GeoDataFrame.from_features(src, crs=src.crs)
 

In [59]:
# empty label list 
labels = gpd.GeoDataFrame(columns=["label", "geometry", "start_year", "end_year"], geometry="geometry")

In [103]:
m = geemap.Map()

def map_polygon(polygon, collection_name, layer_name, yyyymmdd1="2024-01-01", yyyymmdd2="2024-12-29", num_tasks=10):

    # Load in Year Data to Aid in Label Creation
    collection = ee.ImageCollection(collection_name) \
        .filterDate(yyyymmdd1, yyyymmdd2) \
        .filterBounds(polygon) \
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20)) \
        .median() \
        .clip(polygon)
    
    vis = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

    m.addLayer(collection, vis, layer_name)

    # center map on the polygon
    coords = polygon.geometry().centroid().coordinates().getInfo()
    m.setCenter(coords[0], coords[1], 10)

label_dropdown = widgets.Dropdown(
    options=label_list,
    description='Label:',
    value=label_list[0],
)

# Plots Each Year
for year in date_range:
    map_polygon(geemap.geopandas_to_ee(aoi), collection_name, f"Year {year}", yyyymmdd1=f"{year}-01-01", yyyymmdd2=f"{year}-12-29")

date_slider = widgets.SelectionSlider(
    options=[str(year) for year in range(2019, 2025)], # 2019 to 2024 for full years
    value='2019',
    description='Date:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

time_frame_slider = widgets.SelectionSlider(
    options=[str(month) for month in range(1, 13)], # default 12 as its the full year 
    value='3',
    description='timeframe:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

colour_dict = {
    "Urban": "blue",
    "Agricultural": "green",
    "Water": "cyan",
    "Wasteland": "brown",
    "Wetland": "lightblue", 
} # other is grey by default

def append_label_to_points(b):
    """Appends the selected label to the points in the labels GeoDataFrame."""

    global labels

    selected_label = label_dropdown.value
    start_year = int(date_slider.value)
    span = int(time_frame_slider.value)
    end_year = start_year + span - 1

    # Extract drawn features (assumed to be Points)
    all_drawn_features = m.draw_features
    if not all_drawn_features:
        print("No features drawn.")
        return
    
    points = []
    for feature in all_drawn_features:
        try:
            coords = feature.geometry().coordinates().getInfo()  # [lon, lat]
            point = Point(coords)
            points.append(point)
        except Exception as e:
            print("Skipping feature due to error:", e)

    # create dataframe from points
    new_labels = gpd.GeoDataFrame({
        'label': [selected_label] * len(points),
        'start_year': [start_year] * len(points),
        'end_year': [end_year] * len(points),
        'geometry': points
    }, crs="EPSG:4326")

    # Clear All Drawn Features
    m.draw_control.clear()


    # Append to the existing labels GeoDataFrame
    labels = pd.concat([labels, new_labels], ignore_index=True)

    # draw_all_labels()

# def draw_all_labels(layer_name="Labelled Features"):
#     """Draws all labels as one geemap layer, with color per label."""

#     global labels

#     if labels.empty:
#         print("No labels to draw.")
#         return

#     # Assign color based on label
#     colours = labels['label'].map(colour_dict).fillna('gray')

#     # Use geemap's built-in gdf_to_geojson to prepare a styled layer
#     geojson = geemap.gdf_to_geojson(labels)

#     # Plot using custom styling
#     m.add_geojson(
#         geojson,
#         layer_name=layer_name,
#         style={
#             "radius": 5,
#             "color": colours,  # outline color
#             "fillColor": colours,  # from 'color' field in GeoDataFrame
#             "fillOpacity": 0.8,
#         }
#     )

# Label Range  
label_output = widgets.Label()
def update_label(*args):
    start_year = int(date_slider.value)
    span = int(time_frame_slider.value)
    end_year = start_year + span - 1
    label_output.value = f"Selected range: {start_year} to {end_year}"

date_slider.observe(update_label, names='value')
time_frame_slider.observe(update_label, names='value')

update_label() 

# render_btn = widgets.Button(description="Draw Selected Date Layer", position="bottomright") 
# widget_draw = widgets.VBox([render_btn])
# m.add_widget(widget_draw, position="bottomright")

append_btn = widgets.Button(description="Append Label to Points")
append_btn.on_click(append_label_to_points)

# display(widgets.HBox([date_slider, month_slider, time_frame_slider, render_btn]))
display(widgets.VBox([
    widgets.HBox([date_slider, time_frame_slider]),
    widgets.HBox([label_dropdown, label_output, append_btn]),
]))

m

Map(center=[34.22411402596497, 43.91430818231603], controls=(WidgetControl(options=['position', 'transparent_b…

In [110]:
labels.head()

,label,geometry,start_year,end_year,color
0,Urban,POINT (43.88316 34.18889),2019,2023,blue
1,Urban,POINT (43.87088 34.19300),2019,2023,blue
2,Urban,POINT (43.87509 34.20301),2019,2023,blue
3,Urban,POINT (43.89260 34.19570),2019,2023,blue
4,Urban,POINT (43.90302 34.19311),2019,2021,blue


In [105]:
data_dir = pd.DataFrame({
    "dir": [f"data/{year}.csv" for year in range(2019, 2025)],
    "year": [year for year in range(2019, 2025)]
})

In [106]:
data_dir.head()

,dir,year
0,data/2019.csv,2019
1,data/2020.csv,2020
2,data/2021.csv,2021
3,data/2022.csv,2022
4,data/2023.csv,2023


In [135]:
def __reduce_to_common__(labels_df, start_year, end_year):
    """
    Splits the labels DataFrame into two parts:
    1. Common labels that fall within the specified start and end years.
    2. Uncommon labels that do not fall within the specified range, and may be year specific.
    
    Parameters:
        labels_df (GeoDataFrame):   The labels DataFrame to reduce.
        start_year (int):           The starting year for the reduction.
        end_year (int):             The ending year for the reduction.
    
    Returns:
        GeoDataFrame:               The Reduced labels DataFrame.
    """
    common_df = labels_df[(labels_df['start_year'] <= start_year) & (labels_df['end_year'] >= end_year)]
    uncommon_df = labels_df[(labels_df['start_year'] > start_year) | (labels_df['end_year'] < end_year)]
    return common_df.reset_index(drop=True), uncommon_df.reset_index(drop=True)

common_df, uncommon_df = __reduce_to_common__(labels, start_year=2019, end_year=2024)
print(f"Reduced labels DataFrame to {len(common_df)} entries from {len(labels)} original entries.")
print(f"Uncommon labels DataFrame has {len(uncommon_df)} entries.")
print(f"original labels DataFrame has {len(labels)} entries.")
common_df.head()



Reduced labels DataFrame to 6 entries from 17 original entries.
Uncommon labels DataFrame has 11 entries.
original labels DataFrame has 17 entries.


,label,geometry,start_year,end_year,color
0,Water,POINT (43.84651 34.20350),2019,2030,NaN
1,Water,POINT (43.84609 34.19846),2019,2030,NaN
2,Water,POINT (43.85501 34.20020),2019,2030,NaN
3,Water,POINT (43.85149 34.20833),2019,2030,NaN
4,Water,POINT (43.83828 34.22345),2019,2030,NaN


In [ ]:
class ts_cluster():

    yearly_labels = pd.DataFrame(columns=["year_start", "year_end", "label", "geometry"])
    common_labels = pd.DataFrame(columns=["year_start", "year_end", "label", "geometry"])

    start_year = 2019
    end_year = 2024
    def __init__(self, ts_point_labels, data_dir, mapping, start_year, end_year, passes=6):
        """
            Initializes the ts_cluster object with time-series point labels, data directory, mapping, and number of passes.
        """
        
        self.start_year = int(start_year)
        self.end_year = int(end_year)

        if start_year > end_year:
            raise ValueError("start_year must be less than or equal to end_year")
        
        common_labels = self.__reduce_to_common__(ts_point_labels, start_year, end_year)

        # exclude common labels from yearly labels
        yearly_labels = ts_point_labels[~ts_point_labels.index.isin(self.common_labels.index)].copy()
        self.yearly_labels = yearly_labels


    def __reduce_to_common__(self, labels_df):
        """
        Splits the labels DataFrame into two parts:
        1. Common labels that fall within the specified start and end years.
        2. Uncommon labels that do not fall within the specified range, and may be year specific.
        
        Parameters:
            labels_df (GeoDataFrame):   The labels DataFrame to reduce.
            start_year (int):           The starting year for the reduction.
            end_year (int):             The ending year for the reduction.
        
        Returns:
            GeoDataFrame:               The Reduced labels DataFrame.
        """
        common_df = labels_df[(labels_df['start_year'] <= self.start_year) & (labels_df['end_year'] >= self.end_year)]
        uncommon_df = labels_df[(labels_df['start_year'] > self.start_year) | (labels_df['end_year'] < self.end_year)]
        return common_df.reset_index(drop=True), uncommon_df.reset_index(drop=True)

